In [130]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import numpy as np
import tensorflow as tf                  
import tensorflow.nn as nn               # nn contains useful neural network functions like softmax, relu, tanh
from tensorflow import keras             # high level API for TensorFlow
import tensorflow.keras.layers as layers # contains layers like RNN, CNN, LSTM
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [131]:
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")


### Shape of data

In [132]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


### RNN

In [36]:
rnn_clf = keras.Sequential([keras.layers.LSTM(16, input_shape=(1000, 22), return_sequences=True,
                                              dropout=0.5, name="LSTM-1"),
                            keras.layers.LSTM(8, dropout=0.5, name="LSTM-2"),
                            keras.layers.Dense(4, activation=nn.softmax, name="class-probs")
                           ])
rnn_clf.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 LSTM-1 (LSTM)               (None, 1000, 16)          2496      
                                                                 
 LSTM-2 (LSTM)               (None, 8)                 800       
                                                                 
 class-probs (Dense)         (None, 4)                 36        
                                                                 
Total params: 3,332
Trainable params: 3,332
Non-trainable params: 0
_________________________________________________________________


In [60]:
y_train_valid[y_train_valid==769] = 0  # cue onset left
y_train_valid[y_train_valid==770] = 1  # cue onset right
y_train_valid[y_train_valid==771] = 2  # cue onset foot
y_train_valid[y_train_valid==772] = 3  # cue onset tongue

y_test[y_test==769] = 0  # cue onset left
y_test[y_test==770] = 1  # cue onset right
y_test[y_test==771] = 2  # cue onset foot
y_test[y_test==772] = 3  # cue onset tongue

In [96]:
# split the training and validation
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.2, random_state=12345)

In [97]:
# construct the dataset from the NumPy arrays
X_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
X_valid = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
# X_test = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# batch and shuffle
X_train = X_train.shuffle(1024).batch(32)
X_valid = X_valid.shuffle(1024).batch(32)
# X_test = X_test.shuffle(1024).batch(32)

# print(X_train)

In [122]:
# using the built in sparse categorical cross entropy loss
loss_fn = keras.losses.SparseCategoricalCrossentropy()

# choose an optimizer
opt = keras.optimizers.Adam(learning_rate=0.0001)
# there are some other optimizers you can choose from
#opt = keras.optimizers.Adamax()
#opt = keras.optimizers.RMSprop()
#opt = keras.optimizers.SGD()

In [57]:
tf.config.list_physical_devices('GPU')  # if this returns empty enable GPU in collab

[]

In [123]:
rnn_clf.compile(optimizer=opt, loss=loss_fn, metrics='accuracy')
history = rnn_clf.fit(x=X_train, validation_data=X_valid, epochs=10)

Epoch 1/10
53/53 [==============================] - 85s 1s/step - loss: 1.3807 - accuracy: 0.2855 - val_loss: 1.3968 - val_accuracy: 0.2577
Epoch 2/10
53/53 [==============================] - 71s 1s/step - loss: 1.3785 - accuracy: 0.2920 - val_loss: 1.3972 - val_accuracy: 0.2553
Epoch 3/10
53/53 [==============================] - 70s 1s/step - loss: 1.3806 - accuracy: 0.2725 - val_loss: 1.3970 - val_accuracy: 0.2530
Epoch 4/10
53/53 [==============================] - 77s 1s/step - loss: 1.3777 - accuracy: 0.2843 - val_loss: 1.3971 - val_accuracy: 0.2553
Epoch 5/10
53/53 [==============================] - 76s 1s/step - loss: 1.3819 - accuracy: 0.2813 - val_loss: 1.3971 - val_accuracy: 0.2553
Epoch 6/10
53/53 [==============================] - 77s 1s/step - loss: 1.3802 - accuracy: 0.2713 - val_loss: 1.3968 - val_accuracy: 0.2600
Epoch 7/10
53/53 [==============================] - 76s 1s/step - loss: 1.3799 - accuracy: 0.2760 - val_loss: 1.3966 - val_accuracy: 0.2600
Epoch 8/10
53/53 [==

In [103]:
# plt.plot(history.history["accuracy"], label="training accuracy")
# plt.plot(history.history["val_accuracy"], label="validation accuracy")
# plt.legend()
# plt.show()

### CNN

In [124]:
cnn_clf = tf.keras.Sequential(
    [
        keras.layers.Conv2D(6, (50,1), strides=(25,1), input_shape=(1000,22,1), name="conv1"),
        keras.layers.Conv2D(6, (1, 4), strides=(1,2)),
        keras.layers.AveragePooling2D(pool_size=(10,2)),
        keras.layers.Flatten(),
        keras.layers.Dense(32),
        keras.layers.Dense(4, activation=nn.softmax, name="class-probs")
    ])
cnn_clf.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 39, 22, 6)         306       
                                                                 
 conv2d_1 (Conv2D)           (None, 39, 10, 6)         150       
                                                                 
 average_pooling2d_5 (Averag  (None, 3, 5, 6)          0         
 ePooling2D)                                                     
                                                                 
 flatten_5 (Flatten)         (None, 90)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2912      
                                                                 
 class-probs (Dense)         (None, 4)                 132       
                                                      

In [129]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
cnn_clf.compile(optimizer=opt, loss=loss_fn, metrics='accuracy')
history = cnn_clf.fit(x=X_train, validation_data=X_valid, epochs=10)

Epoch 1/10
53/53 [==============================] - 3s 22ms/step - loss: 2.1331 - accuracy: 0.2630 - val_loss: 1.8233 - val_accuracy: 0.3002
Epoch 2/10
53/53 [==============================] - 1s 15ms/step - loss: 1.8320 - accuracy: 0.2671 - val_loss: 1.6509 - val_accuracy: 0.2931
Epoch 3/10
53/53 [==============================] - 1s 15ms/step - loss: 1.6746 - accuracy: 0.2648 - val_loss: 1.5529 - val_accuracy: 0.3097
Epoch 4/10
53/53 [==============================] - 1s 15ms/step - loss: 1.5825 - accuracy: 0.2713 - val_loss: 1.4972 - val_accuracy: 0.3002
Epoch 5/10
53/53 [==============================] - 1s 16ms/step - loss: 1.5248 - accuracy: 0.2766 - val_loss: 1.4607 - val_accuracy: 0.2979
Epoch 6/10
53/53 [==============================] - 1s 15ms/step - loss: 1.4857 - accuracy: 0.2861 - val_loss: 1.4407 - val_accuracy: 0.2955
Epoch 7/10
53/53 [==============================] - 1s 15ms/step - loss: 1.4589 - accuracy: 0.2837 - val_loss: 1.4240 - val_accuracy: 0.2931
Epoch 8/10
53

### CNN with Residual Connections

In [127]:
inputs = keras.Input(shape=(1000,22,1), name='eeg')

block_1_output = keras.layers.Conv2D(6, (50,1), strides=(25,1), name='conv1')(inputs)
block_2_output = keras.layers.Conv2D(6, (1, 4), strides=(1,1), padding='same', name='conv2')(block_1_output)

x = keras.layers.add([block_1_output, block_2_output])
x = keras.layers.AveragePooling2D(pool_size=(10,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(32, name='fully-connected-1')(x)
class_probs = keras.layers.Dense(4, activation=nn.softmax, name='class-probs')(x)

cnn_res_clf = keras.Model(inputs=inputs, outputs=class_probs, name="eeg_resnet")
cnn_res_clf.summary()

Model: "eeg_resnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 eeg (InputLayer)               [(None, 1000, 22, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv1 (Conv2D)                 (None, 39, 22, 6)    306         ['eeg[0][0]']                    
                                                                                                  
 conv2 (Conv2D)                 (None, 39, 22, 6)    150         ['conv1[0][0]']                  
                                                                                                  
 add_4 (Add)                    (None, 39, 22, 6)    0           ['conv1[0][0]',         

In [128]:
opt = keras.optimizers.Adamax()
cnn_res_clf.compile(optimizer=opt, loss=loss_fn, metrics='accuracy')
history = cnn_res_clf.fit(x=X_train, validation_data=X_valid, epochs=10)

Epoch 1/10
53/53 [==============================] - 3s 23ms/step - loss: 2.0564 - accuracy: 0.2896 - val_loss: 1.6298 - val_accuracy: 0.2931
Epoch 2/10
53/53 [==============================] - 1s 17ms/step - loss: 1.5028 - accuracy: 0.3174 - val_loss: 1.4864 - val_accuracy: 0.3310
Epoch 3/10
53/53 [==============================] - 1s 18ms/step - loss: 1.4178 - accuracy: 0.3481 - val_loss: 1.4126 - val_accuracy: 0.3546
Epoch 4/10
53/53 [==============================] - 1s 17ms/step - loss: 1.3723 - accuracy: 0.3593 - val_loss: 1.3885 - val_accuracy: 0.3357
Epoch 5/10
53/53 [==============================] - 1s 17ms/step - loss: 1.3372 - accuracy: 0.3800 - val_loss: 1.3860 - val_accuracy: 0.3641
Epoch 6/10
53/53 [==============================] - 1s 17ms/step - loss: 1.3217 - accuracy: 0.3735 - val_loss: 1.3467 - val_accuracy: 0.3735
Epoch 7/10
53/53 [==============================] - 1s 17ms/step - loss: 1.2938 - accuracy: 0.4119 - val_loss: 1.3424 - val_accuracy: 0.3877
Epoch 8/10
53